In [1]:
#fine tuning SAM2 with Cityscapes dataset
import os, sys
import numpy as np
KD_path = "/home/avalocal/thesis23/KD"
sys.path.append(KD_path)

In [2]:
'''scratch:
  resolution: 1024
  train_batch_size: 1
  num_train_workers: 10
  num_frames: 8
  max_num_objects: 3
  base_lr: 5.0e-6
  vision_lr: 3.0e-06
  phases_per_epoch: 1
  num_epochs: 40'''



'scratch:\n  resolution: 1024\n  train_batch_size: 1\n  num_train_workers: 10\n  num_frames: 8\n  max_num_objects: 3\n  base_lr: 5.0e-6\n  vision_lr: 3.0e-06\n  phases_per_epoch: 1\n  num_epochs: 40'

In [3]:
#dataset 
#images 
#annorations: labelIds
#prompts: bboxes from labelIds (might consider or not)
import os, sys, glob
from PIL import Image
import torch
from torch.utils.data import Dataset
from cityscapes_original import CityscapesSV
import torch.optim as optim
from torch.utils.data import DataLoader
from loss_fns import MultiStepMultiMasksAndIous, sigmoid_focal_loss, dice_loss
from torchvision.transforms import ToTensor

#dataset and dataloaders
train_dataset = CityscapesSV(root_dir="/media/avalocal/T7/pardis/pardis/perception_system/datasets/cityscapes", split='train')
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)

train_batch_size = 4
num_train_workers = 4

# image, instance, ids,  bbox, num_objects, file_name



/home/avalocal/miniconda3/envs/KD/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


2975


In [4]:
from hydra import initialize_config_dir, compose
from omegaconf import DictConfig
import os
#fine-tuning sam2
ckpt_path = '/media/avalocal/T7/pardis/pardis/perception_system/ckpt/sam2.pth'

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import torch.nn as nn

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SAM2Model(nn.Module):   #used for teacher model of depth prediction
    def __init__(self):
        super(SAM2Model, self).__init__()

        # sam2_checkpoint = os.path.join(current_dir, "sam2/checkpoints/sam2.1_hiera_large.pt")
        sam2_checkpoint = "/home/avalocal/thesis23/KD/sam2/checkpoints/sam2.1_hiera_base_plus.pt"
        model_cfg = "sam2.1/sam2.1_hiera_b+.yaml"
        config_dir = "/home/avalocal/thesis23/KD/sam2/sam2/configs"

        with initialize_config_dir(version_base=None, config_dir=config_dir):
            cfg = compose(config_name=model_cfg)
            self.sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=DEVICE)
            self.sam2_predictor = SAM2ImagePredictor(self.sam2_model)
            self.sam2_predictor.model.sam_mask_decoder.train(True)
            self.sam2_predictor.model.sam_prompt_encoder.train(True)
        
    def forward(self, x, points):
        '''
        input: x | should be np.array with size (h,w,3)
        bbox: array w size (n, 4) where n is the number of bounding boxes
        '''

        # print(x.shape, bbox.shape)
        self.sam2_predictor.set_image(x)

        # prompt encoding
        mask_input, unnorm_coords, labels, unnorm_box = self.sam2_predictor._prep_prompts(
            points, None, None, box=None, mask_logits=None, normalize_coords=True)

        if unnorm_coords is not None:
            concat_points = (unnorm_coords, labels)
        else:
            concat_points = None

        if unnorm_box is not None:
            box_coords = unnorm_box.reshape(-1, 2 , 2) #n,4 -> n,2,2
            box_labels = torch.tensor([[2, 3]], dtype=torch.int, device=unnorm_box.device)
            box_labels = box_labels.repeat(unnorm_box.size(0), 1)
            # we merge "boxes" and "points" into a single "concat_points" input (where
            # boxes are added at the beginning) to sam_prompt_encoder
            if concat_points is not None:
                concat_coords = torch.cat([box_coords, concat_points[0]], dim=1)
                concat_labels = torch.cat([box_labels, concat_points[1]], dim=1)
                concat_points = (concat_coords, concat_labels)
            else:
                concat_points = (box_coords, box_labels)

        
        sparse_embeddings, dense_embeddings = self.sam2_predictor.model.sam_prompt_encoder(
            points=concat_points,
            boxes = None,
            masks = mask_input,
        )



        # mask decoder

        batched_mode = (
            concat_points is not None and concat_points[0].shape[0] > 1
        )  # multi object prediction
        high_res_features = [
            feat_level[-1].unsqueeze(0)
            for feat_level in self.sam2_predictor._features["high_res_feats"]
        ]
        low_res_masks, iou_predictions, _, _ = self.sam2_predictor.model.sam_mask_decoder(
            image_embeddings=self.sam2_predictor._features["image_embed"][-1].unsqueeze(0),
            image_pe=self.sam2_predictor.model.sam_prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
            repeat_image=batched_mode,
            high_res_features=high_res_features,
        )

        # Upscale the masks to the original image resolution
        masks = self.sam2_predictor._transforms.postprocess_masks(
            low_res_masks, self.sam2_predictor._orig_hw[-1]
        )

        # print(masks.shape, "this should be output")





        # masks, scores, logits = self.sam2_predictor.predict(
        #                     point_coords=None,
        #                     point_labels=None,
        #                     box=bbox,
        #                     multimask_output=False,
        #                 )
        # print(masks.shape, scores.shape, logits.shape, "this should be output")

        return masks


# criterion = MultiStepMultiMasksAndIous(weight_dict={"loss_mask": 20.0, "loss_dice": 1.0, "loss_iou": 1.0})
# scaler = torch.cuda.amp.GradScaler()
# model


In [5]:
#train loop
from tqdm import tqdm
import torch.nn.functional as F

#finetuning for cityscapes for semantic segmentation

def train(model, train_loader, optimizer, scheduler, DEVICE, epoch):

    model.train()
    running_loss = 0.0
    for i, batch in enumerate(tqdm(train_loader)):

        torch.cuda.empty_cache()
        
        img, _, _, _, _,labels, file_name = batch

        img *= 255.0
        img = F.interpolate(img, size=(1024, 1024), mode='bilinear', align_corners=False) #1, 3, 1024, 2048
        img = img[0].permute(1, 2, 0).numpy().copy().astype(np.uint8)

        labels = F.interpolate(labels, size=(1024, 1024), mode='nearest')
        labels = labels[0].numpy().copy().astype(np.uint8) #1, 1024, 1024
        #1, 1024, 1024 -> N, 1024, 1024
        labels_ = np.zeros((19, 1024, 1024), dtype=np.uint8)
        preds_ = np.zeros((19, 1024, 1024), dtype=np.uint8)
        for i in range(19):
            labels_[i] = (labels==i).astype(np.uint8)

        # print(img.shape, instance.shape, ids.shape, bbox.shape, num_objects, file_name)
        #([1, 3, 1024, 2048]) ([1, 100, 1024, 2048]) ([1, 100]) ([1, 100, 4]) ([19]) ('weimar_000091_000019_leftImg8bit.png',)
        optimizer.zero_grad()

        loss = 0.0


        for cls in range(len(19)):

            #pick N = 5 random points from mask==cls
            mask = labels == cls
            mask = mask.astype(np.uint8)

            #pick N = 5 random points from mask==cls
            points = np.argwhere(mask)
            np.random.shuffle(points)
            points = points[:5]

            pred_mask = model(img, points) #N 1 1024 1024

            if pred_mask.shape[0]>1: 
                #N, 1, 1024, 1024 -> 1, 1, 1024, 1024 
                pred_mask, _ = torch.max(pred_mask, dim=0)
                #pred_mask.max(dim=0, keepdim=True)






            focalLoss = sigmoid_focal_loss(pred_mask, mask, 1, loss_on_multimask=False)
            diceLoss = dice_loss(pred_mask, mask, 1, loss_on_multimask=False)


            loss += 20.0 * focalLoss + 1.0 * diceLoss

        # focalLoss = sigmoid_focal_loss(masks, instance, num_objects[0], loss_on_multimask=False)
        # diceLoss = dice_loss(masks, instance, num_objects[0], loss_on_multimask=False)
        # loss = 20 * focalLoss + 1 * diceLoss

        #backpropagation
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item()

    return running_loss/len(train_loader)

num_epochs = 50

model = SAM2Model()
model = model.to(DEVICE)
##image_encoder, memory_attention, memory_encoder, sam_prompt_encoder, sam_mask_decoder
#freeze all layers except sam_prompt_encoder and sam_mask_decoder
for name, param in model.named_parameters():
    if 'sam_prompt_encoder' in name or 'sam_mask_decoder' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
    
optimizer = optim.AdamW(model.parameters(), lr=5.0e-6, weight_decay=1e-4, betas=(0.9, 0.999))
steps_per_epoch = len(train_loader)
scheduler = optim.lr_scheduler.PolynomialLR(optimizer, steps_per_epoch * num_epochs, power=1.0) #mask2former paper

        

#train loop

for epoch in range(num_epochs):

    torch.cuda.empty_cache()

    train_loss = train(model, train_loader, optimizer, scheduler, DEVICE, epoch)
    print(f'Epoch: {epoch}, Train Loss: {train_loss}')
    






100%|██████████| 2975/2975 [11:52<00:00,  4.18it/s]


Epoch: 0, Train Loss: 35.176852345426546


100%|██████████| 2975/2975 [11:39<00:00,  4.25it/s]


Epoch: 1, Train Loss: 27.81259161644623


100%|██████████| 2975/2975 [11:33<00:00,  4.29it/s]


...

100%|██████████| 2975/2975 [11:41<00:00,  4.24it/s]


Epoch: 3, Train Loss: 19.787373292826803


100%|██████████| 2975/2975 [11:52<00:00,  4.18it/s]


Epoch: 4, Train Loss: 16.219683913663655


  0%|          | 0/2975 [00:00<?, ?it/s]

In [1]:
import os, sys
import numpy as np
KD_path = "/home/avalocal/thesis23/KD"
sys.path.append(KD_path)

In [2]:


import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms.functional as TF
import torch
from model import InstanceStudent
from utils import LargeScaleJittering, instance_inference

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mean = torch.tensor([[[123.6750]], [[116.2800]], [[103.5300]]]).to(DEVICE)
std = torch.tensor([[[58.3950]], [[57.1200]], [[57.3750]]]).to(DEVICE)



img = Image.open("/media/avalocal/T7/pardis/pardis/perception_system/datasets/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png")
img = np.array(img)
# img = cv2.resize(img, (1024, 512))
img = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float()/255.0
print(img.max(), img.min())
img = TF.resize(img, (512, 1024))
img = img.to(DEVICE)



# plt.imshow(img[0].permute(1, 2, 0))


model =InstanceStudent("base_plus")

ckpt = "/home/avalocal/Desktop/checkpoint_epoch_460.pth"
checkpoint = torch.load(ckpt, map_location=DEVICE, weights_only=True)
print(checkpoint['model_state_dict'].keys())
checkpoint['model_state_dict'] = {k.replace('module.', ''): v for k, v in checkpoint['model_state_dict'].items()}
model.load_state_dict(checkpoint['model_state_dict'])

model = model.to(DEVICE)
model.eval()

with torch.amp.autocast(device_type='cuda'):
    pred_out = model(img.to(DEVICE))



tensor(1.) tensor(0.)
odict_keys(['module.image_encoder.trunk.pos_embed', 'module.image_encoder.trunk.pos_embed_window', 'module.image_encoder.trunk.patch_embed.proj.weight', 'module.image_encoder.trunk.patch_embed.proj.bias', 'module.image_encoder.trunk.blocks.0.norm1.weight', 'module.image_encoder.trunk.blocks.0.norm1.bias', 'module.image_encoder.trunk.blocks.0.attn.qkv.weight', 'module.image_encoder.trunk.blocks.0.attn.qkv.bias', 'module.image_encoder.trunk.blocks.0.attn.proj.weight', 'module.image_encoder.trunk.blocks.0.attn.proj.bias', 'module.image_encoder.trunk.blocks.0.norm2.weight', 'module.image_encoder.trunk.blocks.0.norm2.bias', 'module.image_encoder.trunk.blocks.0.mlp.layers.0.weight', 'module.image_encoder.trunk.blocks.0.mlp.layers.0.bias', 'module.image_encoder.trunk.blocks.0.mlp.layers.1.weight', 'module.image_encoder.trunk.blocks.0.mlp.layers.1.bias', 'module.image_encoder.trunk.blocks.1.norm1.weight', 'module.image_encoder.trunk.blocks.1.norm1.bias', 'module.image_enc

In [3]:
pred_out.keys()
# pred_out['pred_masks'] = TF.resize(pred_out['pred_masks'], (1024, 2048),  interpolation=TF.InterpolationMode.NEAREST) 

mask_pred_results = pred_out['pred_masks'] #1, 100, 1024, 2048
mask_cls_results = pred_out['pred_logits'] #1, 100, 9

del pred_out #just for memory

processed_results = []
for mask_cls_result, mask_pred_result in zip(
    mask_cls_results, mask_pred_results
):
    instance_result = instance_inference(mask_cls_result, mask_pred_result)
    processed_results.append(
        {
            # "boxes": instance_result.pred_boxes.tensor,
            "classes": instance_result.pred_classes,
            "masks": instance_result.pred_masks,
            "scores": instance_result.scores,
        }
    )   

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.9359e-05, 0.0000e+00, 0.0000e+00, 1.2201e-05, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0132e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0585e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.3320e-05, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.7560e-06,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0407e-05, 0.0000e+00, 1.0020e-05, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.1330e-05, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.2265e-05, 0.0000e+00, 0.0000e+